### Core Logic

In [47]:
# Imports
import os
import cv2
import tqdm
import imutils
import operator
import numpy as np
import tensorflow as tf
from google.colab import files
import matplotlib.pyplot as plt
from collections import Counter

# Assignments
C_Rows = 'ABCDEFGHI'
C_Cols = '123456789'

# Logic

class Puzzle():
    def __init__(self, filename):
        """Import and read the image."""
        self.puzzle = cv2.imread(filename, 0)
        self.recognizer = C_Recognizer

    def dist(self, p1, p2):
        """Returns the scalar distance between two points"""
        return np.sqrt(((p2[0] - p1[0]) ** 2) + ((p2[1] - p1[1]) ** 2))

    def infer_grid(self, img):
        squares = []
        side = img.shape[:1]
        side = side[0] / 9
        for i in range(9):
            for j in range(9):
                p1 = (i * side, j * side) 
                p2 = ((i + 1) * side, (j + 1) * side) 
                squares.append((p1, p2))
        return squares

    def crop_and_warp(self, img, crop_rect):
        """Crops and warps a rectangular section from an image into a square of similar size."""

        tl, tr, br, bl = crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3]
        src = np.array([tl, tr, br, bl], dtype='float32')
        side = max([self.dist(br, tr), self.dist(tl, bl), self.dist(br, bl), self.dist(tl, tr)])
        dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')
        m = cv2.getPerspectiveTransform(src, dst)
        return cv2.warpPerspective(img, m, (int(side), int(side)))

    def find_corners(self, img):
        """Finds the 4 extreme corners of the largest contour in the image.
            Bottom-Left  : Smallest x - y value
            Bottom-Right : Largest  x + y value
            Top-Left     : Smallest x + y value
            Top-Right    : Largest  x - y value"""

        contours, h = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        polygon = contours[0]

        bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))

        return [polygon[top_left][0], polygon[top_right][0], polygon[bottom_right][0], polygon[bottom_left][0]]


    def preprocess(self, img, if_dilate = False):
        """Gaussian Blur to reduce the noise in the image / smoothen the image. Technically, high frequency components are removed.
                1. The ksize should be an odd number tuple. Directly proportional to blur intensity.
                2. Standard Deviation of the Gaussian kernel.
           Thresholding for image segmentation - adaptive (different threshold for different areas of the image.
                1. The maxValue is dependent on the color format of the image. 
                2. The adaptive method is used to select the threshold.
                3. The thresholdType is used to control the color format of the output.
                4. The blockSize is used to control the smudgeness of the output. Should be odd and greater than one.
                5. The C is the contrast. Directly proportional to brightness."""
        pic = img
        pic = cv2.GaussianBlur(src = pic, ksize = (1, 1), sigmaX = cv2.BORDER_DEFAULT)
        pic = cv2.adaptiveThreshold(src = pic, maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C | cv2.ADAPTIVE_THRESH_MEAN_C,
                                    thresholdType = cv2.THRESH_BINARY_INV, blockSize = 9, C = 2)
        if if_dilate:
            pic = cv2.dilate(src = pic, kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype= np.uint8))
        return pic   

    def predict(self, cell):
        cell = np.array(cell).reshape(1, 28, 28, 1)
        prediction = self.recognizer.predict(cell)
        return np.argmax(prediction)

    def get_digits(self, img, squares):
        """Predict the digit in the cell, after resizing the image to 252 x 252 px, since MNIST dataset contains images of 28 x 28 px."""
        digits = []
        img = self.preprocess(img, if_dilate = False)
        img = cv2.resize(img, (252, 252))
        for i in range(0, 250, 28):
            row = []
            for j in range(0, 250, 28):
                cell = img[i:i+28, j:j+28]
                digit = self.predict(cell)
                row.append(str(digit))
                if j in [56, 140]:
                    row.append('|')
            digits.append('  '.join(row))
            if i in [56, 140]:
                digits.append('-' * 33)
        return digits


    def prepare_puzzle(self):
        processed = self.preprocess(self.puzzle, if_dilate = True)
        corners = self.find_corners(processed)
        cropped = self.crop_and_warp(self.puzzle, corners)
        squares = self.infer_grid(cropped)
        digits = self.get_digits(cropped, squares)
        return digits


class Sudoku():

    def __init__(self):
        pass

    def init_board(self):
        i = 0
        for row in C_Rows:
            for col in C_Cols:
                if C_Unsolved[i] == '0':
                    C_Boxes[row + col] = '123456789'
                else:
                    C_Boxes[row + col] = C_Unsolved[i]
                i += 1
        del C_Boxes['X']

    def board_match(self, X, Y):
        return [r + c for r in X for c in Y]

    def popdigit(self, links, k, p):
        for ele in links[k]:
            C_Boxes[ele] = C_Boxes[ele].replace(p, '')

    def reduce(self, board_lnk, board_box):
        for cell in board_box:
            if len(C_Boxes[cell]) == 1:
                self.popdigit(board_lnk, cell, C_Boxes[cell])

    def unique(self, board_seg):
        for seg in board_seg:
            fixnum = ''
            unqcnt = 0
            string = ''.join([C_Boxes[cell] for cell in seg if len(C_Boxes[cell]) != 1])
            stringcnts = Counter(string)
            if len(stringcnts) > 2:
                for k1, v1 in stringcnts.items():
                    if v1 == 1:
                        unqcnt += 1
                        fixnum += k1
            if unqcnt == 1:
                for cell in seg:
                    if fixnum in C_Boxes[cell]:
                        C_Boxes[cell] = fixnum

    def evaluate(self, board_seg):
        sums = 0
        try:
            for seg in board_seg:
                for cell in seg:
                    sums += int(C_Boxes[cell])
            if sums == 405:
                return True
            else:
                return False
        except:
            return False

    def join_board(self):
        solved_board = ''
        for row in C_Rows:
            for col in C_Cols:
                solved_board += C_Boxes[row + col]
        return solved_board

    def solve(self):
        board_box = self.board_match(C_Rows, C_Cols)
        board_row = [self.board_match(R, C_Cols) for R in C_Rows]
        board_col = [self.board_match(C_Rows, C) for C in C_Cols]
        board_sqr = [self.board_match(Rs, Cs) for Rs in ('ABC', 'DEF', 'GHI') for Cs in ('123', '456', '789')]
        board_lst = board_row + board_col + board_sqr
        board_all = dict((cell, [lst for lst in board_lst if cell in lst]) for cell in board_box)
        board_lnk = dict((cell, sorted(set(sum(board_all[cell], [])) - set([cell]))) for cell in board_box)
        for i in range(5):
            for j in range(3):
                self.reduce(board_lnk, board_box)
            for board_seg in [board_sqr, board_col, board_row]:
                self.unique(board_seg)
        if self.evaluate(board_sqr) and self.evaluate(board_col) and self.evaluate(board_row):
            a = self.join_board()
            return a
        else:
            return None

    def display_board(self):
        print('-' * 37)
        for row in C_Rows:
            board = '|'
            for col in C_Cols:
                board += '  ' + C_Boxes[row + col]
                if col in '369':
                    board += '  |'
            print(board)
            if row in 'CF':
                print('-' * 37)
        print('-' * 37)
        


### Validation of Inputs

In [31]:
C_Root = '/content/drive/MyDrive/Sudoku'
print("Loading the DigitRecognizer Model...")
try:
    C_Recognizer = tf.keras.models.load_model(f"{C_Root}/DigitRecognizer")
    C_Proceed = True
    print("DigitRecognizer Model Successfully Loaded!")
except:
    C_Proceed = False
    print("DigitRecognizer Model not loaded. Cannot proceed further!")

if C_Proceed:
    uploads = files.upload()
    [C_Filename] = uploads.keys()

Loading the DigitRecognizer Model...
DigitRecognizer Model Successfully Loaded!


Saving Sudoku.jpg to Sudoku (1).jpg


### Extract Sudoku Board from the image

In [32]:
board = Puzzle(C_Filename)
C_Unsolved = board.prepare_puzzle()
print("2-D Board\n")
print('\n'.join(C_Unsolved))
oneline = ''
for row in C_Unsolved:
    for char in row:
        if char in '0123456789':
            oneline += char
    oneline += '.'
print("\nSingle Line Version\n")
print(oneline)

2-D Board

0  0  0  |  6  0  0  |  3  0  0
7  0  8  |  0  0  0  |  0  0  9
0  0  0  |  0  0  5  |  0  8  0
---------------------------------
0  7  0  |  0  0  0  |  0  0  3
8  0  0  |  0  0  0  |  0  0  5
0  0  0  |  0  0  0  |  0  7  0
---------------------------------
0  0  0  |  2  0  0  |  0  0  0
3  0  0  |  0  0  0  |  2  0  8
0  0  2  |  2  0  0  |  0  0  0

Single Line Version

000600300.708000009.000005080..070000003.800000005.000000070..000200000.300000208.002200000.


### Check correctness of inferred board and begin solving

If the board shown above is not the correct one,


1.   Use the single-line version of the board to edit.
2.   Assign the edited single-line version of the board to C_Unsolved.
3.   Run the following cell to solve the sudoku.
4.   Remember to uncomment the C_Unsolved line :)

If it is correct, no actions are necessary. Just run this cell.

In [53]:
C_Unsolved = '600120384008459072000006005000264030070080006940003000310000050089700000502000190'
C_Unsolved = ''.join([char for char in C_Unsolved if char != '.'])
game = Sudoku()
C_Boxes = {'X' : 0}
game.init_board()
A = game.solve()
if A:
    game.display_board()
else:
    print("Solution not identified")

-------------------------------------
|  6  9  5  |  1  2  7  |  3  8  4  |
|  1  3  8  |  4  5  9  |  6  7  2  |
|  7  2  4  |  8  3  6  |  9  1  5  |
-------------------------------------
|  8  5  1  |  2  6  4  |  7  3  9  |
|  2  7  3  |  9  8  1  |  5  4  6  |
|  9  4  6  |  5  7  3  |  8  2  1  |
-------------------------------------
|  3  1  7  |  6  9  2  |  4  5  8  |
|  4  8  9  |  7  1  5  |  2  6  3  |
|  5  6  2  |  3  4  8  |  1  9  7  |
-------------------------------------
